In [1]:
import tensorflow as tf
import numpy as np
import sys
import os
import functools
import time
import config
import shutil



#To change
from my_load_data import load_dataset_fn 
print("loaded libs")

loaded libs


In [2]:
print (len(os.listdir(config.PIC_SRC_DIR)))


params = {"result_dir": config.RESULT_DIR,
          "learning_rate": config.LR,
          "img_size": config.CNN_IN_HEIGHT,
          "num_channels":config.CNN_IN_CH,
          "num_classes": config.CNN_OUTPUT_SIZE,
          "batch_size": config.BATCH_SIZE,
          "total_steps": 1,
          "keep_probability": config.KEEP_PROB,
          "print_nth_step": 10}
print("loaded params")

1000
loaded params


In [9]:
#WRAPPERS FOR MODEL
def doublewrap(function):
    """
    A decorator of decorator, allowing use of lazy property if no arguments are provided
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator

@doublewrap
def define_scope(function, scope = None, *args, **kwargs):
    """
    Lazy decorator, optimizes code by loading class Model parts only once to memory
    Also its groups tf.Graph, in tensorboard into smaller, more readable parts
    """    
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            #Sorting Graph by Var_scope
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
                print("Initialized Model.{}".format(name))
        return getattr(self, attribute)
    
    return decorator


class Model(object):
    """
    Model of NN
    """
    #Variable of model
    #accuracy = tf.reduce
    
    
### INITIALIZATION ####    
    def __init__(self, params):
        #self.permutation = np.random.permutation(range(len(os.listdir(config.PIC_SRC_DIR))))
        
        self.img_size = params["img_size"]
        self.num_channels = params["num_channels"]
        self.num_classes = params["num_classes"]
        self.lr = params["learning_rate"]
        self.keep_prob = params["keep_probability"]
        self.write_step = params["print_nth_step"]
        self.is_training = False
        self.global_step = tf.train.get_or_create_global_step()
        
        #remove bellow
        self.data = tf.placeholder(dtype= tf.float32, shape=[None,256,256,3])
        self.target = tf.placeholder(dtype = tf.float32, shape=[None,5])
        self.permutation = np.random.permutation(range(1,6))
        self.data_iter = 1 
        self.total_epoch = 0
        
        #FUNCTIONS, don not remove
        self.load_data
        self.prediction
        self.loss_op
        self.optimize
        self.summary

    

        
#FUNCTION DEFINITIONS
    def data_pipeline(self):
        """
        loading of TFRecords
        """
        pass
    
    
    
#CHANGE LOAD DATA TO DATA_PIPELINE
    @define_scope    
    def load_data(self):
        """
        Serve random data each iteration
        """
        with tf.name_scope('Input'):
            self.data, self.target = (load_dataset_fn(self.permutation[self.data_iter]))
            #print(type(self.data))
            #print(self.data.shape)
            #print(self.target.shape)
            #self.train_init = iterator.make_initializer(train_data)  # initializer for train_data
            #self.test_init = iterator.make_initializer(test_data)    # initializer for train_data
            
            
##### NEURAL NETWORK #####   
    @define_scope
    def prediction(self, if_training=True):
        """
        Main body of neural network, takes data and labels as input,
        returns feature map of photo
        """
        #INPUT LAYER
        input_layer = tf.reshape(self.data,[-1, self.img_size, self.img_size, self.num_channels]) 
        
        #1 conv layer
        conv1 = tf.layers.conv2d(inputs = self.data, 
                             filters = 32,
                             kernel_size = 5,
                             strides = 1,
                             padding = "same",
                             activation = tf.nn.relu)
        
        #1 pool layer, img size reduced by 1/4
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")

        #2 conv layer
        conv2 = tf.layers.conv2d(inputs = pool1, 
                             filters = 64,
                             kernel_size = 5,
                             strides = 1,
                             padding = "same",
                             activation = tf.nn.relu)

        #2 pool overal image size reduced totaly by factor of 1/16
        pool2 = tf.layers.max_pooling2d(inputs = conv2,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")
        
        pool2_flat = tf.reshape(pool2,[-1, 64*64*64])

        dense = tf.layers.dense(inputs = pool2_flat,
                            units = 128,
                            activation = tf.nn.relu)
        dropout = tf.layers.dropout(dense,rate = self.keep_prob,training = self.is_training)
        
        dense2 = tf.layers.dense(inputs = dropout,
                             units = self.num_classes,
                             activation = tf.nn.relu)
       
        return tf.nn.softmax(dense2)
    
    
    
#### LOSS ####    
    @define_scope
    def loss_op(self):
        """
        loss
        """
        self.loss = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits_v2(logits = self.prediction,
                                                                labels = self.target))
        
        return self.loss
    
    
    
##### OPTIMIZER #####    
    @define_scope    
    def optimize(self):
        """
        Optimizer of model
        """
        optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
        
        return optimizer.minimize(self.loss, global_step=self.global_step)
    
    
    
###summary####    
    @define_scope
    def summary(self):
        '''
        Create summaries to write on TensorBoard
        '''
        with tf.name_scope('summaries'):
            tf.summary.scalar('loss', self.loss)
            tf.summary.histogram('histogram_loss', self.loss)
            self.summary_op = tf.summary.merge_all()
            
            
            
#### TRAIN ###        
    def train(self, sess, ckpt_dir, writer, saver, step, epoch):
        """
        Training op for model
        """
        start_time = time.time()
        self.is_training = True
        total_loss = 0
        num_batches = 0
        self.training = True
        try :
                #train
                loss, _,summary = sess.run([self.loss_op, self.optimize, self.summary_op])
                writer.add_summary(summary, global_step=step)
                if (step + 1) % self.write_step == 0:
                    print('Loss at step {0}: {1}'.format(step, loss))
                    saver.save(sess, ckpt_dir, global_step = step)
                #save
                # writer.add_summary(summary, global_step=step)
                self.data_iter+=1
                step += 1
                total_loss += loss
                num_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss at epoch {0}: {1}'.format(epoch, total_loss/num_batches))
        print('Took: {0} seconds'.format(time.time() - start_time))
        
        return step
    
    
    
 #### EVALUATE ####    
    def evaluate(self, init, step, epoch):
        """
        Evaluate once from test
        """
        start_time = time.time()
        self.is_training = False
        try:
            loss, summary = sess.run([self.loss, self.summary_op])
            writer.add_summary(summary, global_step=step)
        except tf.errors.OutOfRangeError:
            pass
        print('Loss at epoch {0}: {1}'.format(epoch, loss))
        print('Took: {0} seconds'.format(time.time() - start_time))
        
        
        
 ### TO DO ####   
    def train_n_times(self, n_times):
        pass
        
    

In [10]:
total_epoch = 7
def main(remove_results = False):
    result_dir = config.RESULT_DIR
    
    if remove_results == True:
        shutil.rmtree(result_dir, ignore_errors=True)
    #Check for dirs, if not present make them
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    ckpt_dir=os.path.join(result_dir,"ckpt")
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    
    graph = tf.Graph()
    #Get name as default graph
    with graph.as_default():

        print("Starting Session")
        #Assign name to session, assign it's default graph as graph
        with tf.Session(graph=graph) as sess:
            
            #Creating summary writer 
            writer = tf.summary.FileWriter(ckpt_dir, graph=graph)
                
            #Initialization of Model, load all Model functions returning variables
            model = Model(params)
            
            #Assign Initializer
            init = tf.global_variables_initializer()
            
            #Creating save for model session for future saving and restoring model
            saver = tf.train.Saver()
            
            sess.run(init)
            #Loading last checkpoint
            ckpt = tf.train.get_checkpoint_state(result_dir)
            if ckpt and ckpt.model_checkpoint_path:
                #if ckpt found load it and load global step
                saver.restore(sess, ckpt.model_checkpoint_path)
                print("Found checkpoint")
            step = model.global_step.eval()    
                
            print("we are at step {0}".format(step))        
            #Training
            print("Starting Training")

            for epoch in range(1,total_epoch):
                try: 
                    step = model.train(sess, ckpt_dir, writer, saver, step, epoch)
                except KeyboardInterrupt:
                    print("keyboard interrupt")
                    step = model.global_step.eval()
                    break
            #model.global_step = tf.convert_to_tensor(step, dtype=tf.int32)
            saver.save(sess, ckpt_dir, global_step = step)
            print(model.global_step.eval(), step)
        print("Finnished session")
        #Merge all summaries
        #writer.flush()
        writer.add_graph(graph)
        writer.close()
        print("Closed summary, work finnished")

In [11]:
### tf.reset_default_graph()
if __name__ == '__main__':
    check = True
    main(remove_results=check)
    check = False
    main(remove_results=check)

Starting Session
Initialized Model.load_data
Initialized Model.prediction
Initialized Model.loss_op
Initialized Model.optimize
Initialized Model.summary
we are at step 0
Starting Training
Average loss at epoch 1: 19.025510787963867
Took: 5.471312999725342 seconds
Average loss at epoch 2: 19.043148040771484
Took: 4.358249187469482 seconds
Average loss at epoch 3: 18.963726043701172
Took: 4.388251066207886 seconds
Average loss at epoch 4: 18.961332321166992
Took: 4.302246332168579 seconds
Average loss at epoch 5: 18.964054107666016
Took: 4.333247661590576 seconds
Average loss at epoch 6: 18.964054107666016
Took: 4.3522491455078125 seconds
6 6
Finnished session
Closed summary, work finnished
Starting Session
Initialized Model.load_data
Initialized Model.prediction
Initialized Model.loss_op
Initialized Model.optimize
Initialized Model.summary
INFO:tensorflow:Restoring parameters from results\ckpt-6
Found checkpoint
we are at step 6
Starting Training
Average loss at epoch 1: -8.518839836120